# Implementing a 4 parameter logistic curve in Python
Two websites I found that look helpful.
http://people.duke.edu/~ccc14/pcfb/analysis.html
This links to a nice python implementation of 4pl that I will see if I can get to work.
https://www.aphis.usda.gov/animal_health/vet_biologics/publications/STATWI0005.pdf
This odd link had a great explanation of what is going on in a 4pl implementation. This is what really made it start clicking for me.

In [1]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy.optimize import leastsq

In [2]:
def logistic4(x, A, B, C, D):
    """4PL logistic equation."""
    return ((A - D)/(1.0 + ((x / C)**B))) + D

The parameters in a 4PL curve are the following:
* A and D are the upper and lower asymptotes. Since I am thinking in terms of ELISA assays, which one is upper or lower depends on whether it is a competitive assay or not.
* B is the scale factor. I basically think of this as the approximate slope at the inflection point. This is an imperfect way to think about it, but it gets the job done.
* C is the location parameter. This is where the inflection point between the two asymptotes is at.
* x would be, in the case of an ELISA, the concentration parameter.

In [3]:
def residuals(p, y, x):
    """Deviations of data from fitted 4PL curve"""
    A, B, C, D = p
    err = y-logistic4(x, A, B, C, D)
    return err

In [ ]:
def peval(x, p):
    """Evaluated value at x with current parameters."""
    A, B, C, D = p
    return logistic(x, A, B, C, D)